# Data audit

This notebook is part of the supplementary material for AutomatedFishID publication. The purpose of this Jupyter Notebook is for exploring the publicly available dataset.

MIT License

Copyright (c) 2022 AutomatedFishID

Author: Sawitchaya Tippaya

In [2]:
import sys
import os

import json
import pandas as pd
import numpy as np
import glob, random, os, shutil
from matplotlib import pyplot as plt
import matplotlib.patches as patches

import plotly
import plotly.express as px
import plotly.graph_objects as go

import pickle

import cv2
from PIL import Image

import itertools

from fish_utils import *

import pickle

from tqdm import tqdm
from time import sleep

from itertools import product

%matplotlib inline  

# Ozfish Dataset

## Step 1: Ozfish Frames

- EventMeasure measurement, Fish bbox, species, genus, family
- Image source: https://data.pawsey.org.au/public/?path=/FDFML/frames
- Label source: https://data.pawsey.org.au/download/FDFML/metadata/frame_metadata.csv
- top left and bottom right pixel location (need to convert to rectangle at least)
- 64385 unique images

In [3]:
df_meta = pd.read_csv("../datasets/df_meta.csv")
df_meta.fillna("", inplace=True)
df_meta.head()

,uid,file_name,x0,y0,x1,y1,family,genus,species,rect.width,rect.height
0,1,A000001_L.avi.5107.png,806,371,922,448,Scaridae,Chlorurus,capistratoides,116,77
1,2,A000001_R.avi.4902.png,1388,355,1508,435,Scaridae,Chlorurus,capistratoides,120,80
2,3,A000001_L.avi.5174.png,912,280,994,335,Scaridae,Chlorurus,capistratoides,82,55
3,4,A000001_R.avi.4969.png,1600,255,1682,310,Scaridae,Chlorurus,capistratoides,82,55
4,5,A000001_L.avi.5194.png,811,472,979,584,Acanthuridae,Acanthurus,leucocheilus,168,112


In [4]:
print("Total video frames (unique):", df_meta.file_name.nunique())
print("Total Fish Objects (Bounding Boxes):", df_meta.shape[0])
print("Total Fish Family:", df_meta.family.nunique())
print("Total Fish Genus:", df_meta.genus.nunique())
print("Total Fish Species:", df_meta.species.nunique())

Total video frames (unique): 64385
Total Fish Objects (Bounding Boxes): 80983
Total Fish Family: 75
Total Fish Genus: 215
Total Fish Species: 497


## Step 2: Fish measurement files (Point and Labelled)

- Event measure
- Pixel locations for nose and tail of fish which were measured, and a measurement in mm for the given fish
- No bounding box for any fish
- Image source: https://data.pawsey.org.au/public/?path=/FDFML/labelled/frames
- Label source: https://data.pawsey.org.au/public/?path=/FDFML/labelled/measurementfiles
- 32817 unique images

In [5]:
df_fish_measure = pd.read_csv("../datasets/df_fish_measure.csv", low_memory=False)

In [6]:
df_fish_measure.head()

,Activity,Attribute10,Attribute9,Code,Comment,Direction,Family,FilenameLeft,FilenameRight,FrameLeft,...,Precision,RMS,Range,Rx,Ry,Species,Stage,Time,Unnamed: 0,VertDir
0,Passing,NaN,NaN,37386041.0,NaN,6.69932,Scaridae,A000001_L.avi,A000001_R.avi,5107,...,11.56258,2.36216,5029.54699,1487.01031,375.13402,capistratoides,AD,3.40467,0,27.70218
1,Passing,NaN,NaN,37386041.0,NaN,6.69932,Scaridae,A000001_L.avi,A000001_R.avi,5107,...,11.56258,2.36216,5029.54699,1410.92784,416.57732,capistratoides,AD,3.40467,1,27.70218
2,Passing,NaN,NaN,37386041.0,NaN,11.65616,Scaridae,A000001_L.avi,A000001_R.avi,5174,...,61.59869,3.94426,7253.88433,1660.20619,273.40206,capistratoides,AD,3.44933,2,11.32529
3,Passing,NaN,NaN,37386041.0,NaN,11.65616,Scaridae,A000001_L.avi,A000001_R.avi,5174,...,61.59869,3.94426,7253.88433,1622.47423,292.57732,capistratoides,AD,3.44933,3,11.32529
4,Passing,NaN,NaN,37437053.0,NaN,3.77737,Acanthuridae,A000001_L.avi,A000001_R.avi,5194,...,2.83090,0.54547,3132.21538,1367.67010,512.31959,leucocheilus,AD,3.46267,4,5.75408


In [7]:
print("Total Labelled Objects (unique):", df_fish_measure.shape[0])
print("Total video frames (unique):", df_fish_measure.OzFishFrame.nunique())
print("Total Fish Family:", df_fish_measure.Family.nunique())
print("Total Fish Genus:", df_fish_measure.Genus.nunique())
print("Total Fish Species:", df_fish_measure.Species.nunique())

Total Labelled Objects (unique): 83122
Total video frames (unique): 32817
Total Fish Family: 74
Total Fish Genus: 215
Total Fish Species: 501


## Step 3: SPECIES

- Bounding Box Species Annotations
- the Sagemaker Ground Truth Platform combined with VGG annotator done by an ecologist, species
- Image source: https://data.pawsey.org.au/public/?path=/FDFML/labelled/frames
- Label source: https://data.pawsey.org.au/public/?path=/FDFML/labelled/speciesboxes
- 1751 unique images

In [8]:
'''Load species json file'''

species_all = pd.DataFrame()
for i in range(1,4):    
    label = '../datasets/ozfish_speciesboxes/batch0%d_species.json'%(i)
    print(label)   
    
    species = pd.read_json(label)
    species = species[species['_via_img_metadata'].isnull() == False]
    species = pd.json_normalize(species['_via_img_metadata'])

    # extract video which contain regions attibutes
    label_species = species[species["regions"].str.len() != 0]
    
    species_all = pd.concat([species_all, label_species], axis =0)

species_all = species_all.reset_index(drop=True)

'''Convert JSON to a simple table'''

df_species = pd.DataFrame()
for file in species_all.filename:
    df = species_all[species_all.filename == file]
    regions = pd.json_normalize(df.regions.values[0])
    filename = pd.concat([df[['filename', 'size']]]*len(regions), ignore_index=True)
    df2 = pd.concat([filename, regions], axis=1)
    
    df_species = pd.concat([df_species, df2], ignore_index=True)
    
print('Number of video frames with species labelled: ', df_species.filename.nunique())  
df_species.head()

../datasets/ozfish_speciesboxes/batch01_species.json
../datasets/ozfish_speciesboxes/batch02_species.json
../datasets/ozfish_speciesboxes/batch03_species.json
Number of video frames with species labelled:  1751


,filename,size,shape_attributes.name,shape_attributes.x,shape_attributes.y,shape_attributes.width,shape_attributes.height,region_attributes.label
0,A000001_L.avi.17418.png,1747414.0,rect,44.5,177.5,73.5,48.5,Naso sp
1,A000001_L.avi.17418.png,1747414.0,rect,153.5,19.0,280.5,207.0,Kyphosus cinerascens
2,A000001_L.avi.17418.png,1747414.0,rect,447.0,118.0,11.0,4.0,Fish
3,A000001_L.avi.17418.png,1747414.0,rect,619.0,353.5,128.5,89.0,Lutjanus bohar
4,A000001_L.avi.17418.png,1747414.0,rect,951.5,45.5,64.5,100.5,Lutjanus bohar


In [9]:
print("Total video frames (unique):", df_species.filename.nunique())
print("Total Fish Objects (Bounding Boxes):", df_species.shape[0])
print("Total Fish Species:", df_species["region_attributes.label"].nunique())

Total video frames (unique): 1751
Total Fish Objects (Bounding Boxes): 42685
Total Fish Species: 194


In [10]:
df_species[["region_attributes.label"]].value_counts()

region_attributes.label
Fish                       35689
Lethrinus punctulatus       2170
Lutjanus sebae               724
Lutjanus vitta               498
Lethrinus atkinsoni          425
                           ...  
Epibulus insidiator            1
Elagatis bipinnulata           1
Pomacanthus sexstriatus        1
Diagramma sp                   1
Priacanthus hamrur             1
Length: 194, dtype: int64

## Step 4: Relabelled and select only top 12 species (Bridget data)
- Bounding box of every fish in image
- If cannot identify species, label as fish

In [12]:
fish = pd.read_json('../datasets/master-top12.json')
fish = fish[fish['_via_img_metadata'].isnull() == False]
fish = pd.json_normalize(fish['_via_img_metadata'])
fish = fish[fish.regions.str.len() != 0]

# convert JSON to table
df_top12 = pd.DataFrame()
for f in fish.filename:
    df = fish[fish.filename == f]    
    regions = pd.json_normalize(df.regions.values[0])
    info = pd.concat([df[['filename', 'size']]]*len(regions), ignore_index=True)
    df2 = pd.concat([info, regions], axis=1)
    
    df_top12 = pd.concat([df_top12, df2], ignore_index=True)

df_top12["region_attributes.label"].fillna("fish", inplace=True)
df_top12["region_attributes.label"] = df_top12["region_attributes.label"].str.replace(" ","_")
df_top12["region_attributes.label"] = df_top12["region_attributes.label"].str.lower()
df_top12['fish'] = 0

In [13]:
df_top12.head()

,filename,size,shape_attributes.name,shape_attributes.x,shape_attributes.y,shape_attributes.width,shape_attributes.height,region_attributes.label,fish
0,A000001_L.avi.17418.png,1747414,rect,44.5,177.5,73.5,48.5,fish,0
1,A000001_L.avi.17418.png,1747414,rect,153.5,19.0,280.5,207.0,fish,0
2,A000001_L.avi.17418.png,1747414,rect,447.0,118.0,11.0,4.0,fish,0
3,A000001_L.avi.17418.png,1747414,rect,619.0,353.5,128.5,89.0,lutjanus_bohar,0
4,A000001_L.avi.17418.png,1747414,rect,951.5,45.5,64.5,100.5,lutjanus_bohar,0


In [14]:
print("Total video frames (unique):", df_top12.filename.nunique())
print("Total Fish Objects (Bounding Boxes):", df_top12.shape[0])
print("Total Fish Species:", df_top12["region_attributes.label"].nunique())

Total video frames (unique): 1751
Total Fish Objects (Bounding Boxes): 42685
Total Fish Species: 13


In [15]:
df_species[["region_attributes.label"]].value_counts()

region_attributes.label
Fish                       35689
Lethrinus punctulatus       2170
Lutjanus sebae               724
Lutjanus vitta               498
Lethrinus atkinsoni          425
                           ...  
Epibulus insidiator            1
Elagatis bipinnulata           1
Pomacanthus sexstriatus        1
Diagramma sp                   1
Priacanthus hamrur             1
Length: 194, dtype: int64

## Step 5 Data split

### 5.1 Convert Rectangle to YOLO bounding box supported format
- use https://github.com/tensorturtle/rebox  
- `mamba install rebox`
- `rect_to_yolo` function to convert rectangle to yolo format

In [ ]:
def rect_to_yolo(label, x, y, w, h):
    ''' function to suppoer lambda row in pandas'''
    coco_bbox = BBox([x, y, w, h], coco)
    xc, yc, w_, h_ = coco_bbox.as_format(yolo, 1920, 1080).value
    return [label, xc, yc, w_, h_]

In [16]:
from rebox import BBox
from rebox.formats import yolo, coco
from sklearn import preprocessing

df_top12['yolo_fish'] = df_top12.apply(lambda row : rect_to_yolo(row['fish'],
                                                            row['shape_attributes.x'],
                                                            row['shape_attributes.y'],
                                                            row['shape_attributes.width'],
                                                            row['shape_attributes.height']
                                                           ), axis = 1)

le = preprocessing.LabelEncoder()
df_top12['label'] = le.fit_transform(df_top12["region_attributes.label"].values)
df_top12[['label', "region_attributes.label"]].value_counts().sort_index()

label  region_attributes.label 
0      abalistes_stellatus           173
1      acanthurus_triostegus         230
2      epinephelus_areolatus         191
3      epinephelus_multinotatus      118
4      fish                        37573
5      lethrinus_atkinsoni           425
6      lethrinus_punctulatus        2170
7      lutjanus_bohar                130
8      lutjanus_sebae                724
9      lutjanus_vitta                498
10     pentapodus_porosus            121
11     pomacentrus_coelestis         182
12     thalassoma_lunare             150
dtype: int64

### 5.2 Create datasets

1) top-twelve-catch-all: top 12 species, fish. (n_class=13)
2) top-twelve-no-catch-all: top 12 species. (n_class=12)
3) fish-only: fish (n_class=1)


Directory structure

```
Parent
  |__datasets
  |     |__fish
  |          |__top-twelve-catch-all
  |          |    |__images
  |          |    |__labels
  |          |    |__train.txt
  |          |    |__test.txt
  |          |    |__valid.txt
  |          |__top-twelve-no-catch-all
  |          |    |__images
  |          |    |__labels
  |          |    |__train.txt
  |          |    |__test.txt
  |          |    |__valid.txt
  |          |__fish-only
  |               |__images
  |               |__labels
  |               |__train.txt
  |               |__test.txt
  |               |__valid.txt
  |__...
```

### 5.3 Split data

In [17]:
df_top12.head()

,filename,size,shape_attributes.name,shape_attributes.x,shape_attributes.y,shape_attributes.width,shape_attributes.height,region_attributes.label,fish,yolo_fish,label
0,A000001_L.avi.17418.png,1747414,rect,44.5,177.5,73.5,48.5,fish,0,"[0, 0.0423, 0.1868, 0.0383, 0.0449]",4
1,A000001_L.avi.17418.png,1747414,rect,153.5,19.0,280.5,207.0,fish,0,"[0, 0.153, 0.1134, 0.1461, 0.1917]",4
2,A000001_L.avi.17418.png,1747414,rect,447.0,118.0,11.0,4.0,fish,0,"[0, 0.2357, 0.1111, 0.0057, 0.0037]",4
3,A000001_L.avi.17418.png,1747414,rect,619.0,353.5,128.5,89.0,lutjanus_bohar,0,"[0, 0.3559, 0.3685, 0.0669, 0.0824]",7
4,A000001_L.avi.17418.png,1747414,rect,951.5,45.5,64.5,100.5,lutjanus_bohar,0,"[0, 0.5124, 0.0887, 0.0336, 0.0931]",7


- "Pylabel" data annotation tool https://github.com/pylabel-project/pylabel/blob/dev/pylabel/splitter.py

In [18]:
# df2 = StratifiedGroupShuffleSplit(df_top12, train_pct=0.8, test_pct=0.1, val_pct=0.1, weight=0.01, group_col = 'filename', cat_col = 'region_attributes.label', batch_size=16)

# print(df2.groupby(['split'])['filename'].nunique().to_frame()['filename']/(df2.filename.nunique()))

# for f in df2.filename.unique():
    
#     if df2[df2.filename == f]['split'].nunique() != 1:
#         print('something is not right here:', f)

# df_fig1 = ShowClassSplits(df2, normalize=False, cat_col = 'region_attributes.label')
# df_fig1 = df_fig1.reset_index()

### 66-17-17 Split

- Note: the previous CSV file was written out with tuple so it required `ast.literal_eval` when read back using `pd.read_csv` to have a correct format.

In [19]:
SPLIT = "66-17-17"
import ast
df_top12_split_final = pd.read_csv(f"../datasets/fish/2022-05-04_df_top12_split_{SPLIT}.csv", index_col=0, converters={"yolo_fish": ast.literal_eval})
df_top12_split_final.head()

,filename,size,shape_attributes.name,shape_attributes.x,shape_attributes.y,shape_attributes.width,shape_attributes.height,region_attributes.label,fish,label,yolo_fish,split
0,G000039_L.avi.36953.png,1762502,rect,1861.5,514.5,24.5,26.5,fish,0,4,"(0, 0.9759, 0.4887, 0.0128, 0.0245)",train
1,G000039_L.avi.36953.png,1762502,rect,1083.0,283.0,33.0,38.0,fish,0,4,"(0, 0.5727, 0.2796, 0.0172, 0.0352)",train
2,G000039_L.avi.36953.png,1762502,rect,1675.0,233.5,21.0,42.0,fish,0,4,"(0, 0.8779, 0.2356, 0.0109, 0.0389)",train
3,G000039_L.avi.36953.png,1762502,rect,307.5,184.5,198.5,103.0,fish,0,4,"(0, 0.2118, 0.2185, 0.1034, 0.0954)",train
4,G000039_L.avi.36953.png,1762502,rect,1865.0,400.5,26.5,26.0,fish,0,4,"(0, 0.9783, 0.3829, 0.0138, 0.0241)",train


In [20]:
df_top12_split_final.split.value_counts()

train    34141
val       4280
test      4264
Name: split, dtype: int64

In [21]:
for v in ['A0', 'B0', 'E0', 'G0']:
    print(f'{v} total frames', df_top12_split_final[df_top12_split_final.filename.str.contains(v)].filename.nunique())

A0 total frames 342
B0 total frames 0
E0 total frames 266
G0 total frames 1143


In [22]:
for f in df_top12_split_final.filename.unique():
    
    if df_top12_split_final[df_top12_split_final.filename == f]['split'].nunique() != 1:
        print('something is not right here:', f)

df_fig1 = ShowClassSplits(df_top12_split_final, normalize=False, cat_col = 'region_attributes.label')
df_fig1 = df_fig1.reset_index()
df_fig1["scientific_name"] = ['Fish', 'L. punctulatus', 'L. sebae', 'L. vitta', 'L. atkinsoni', 
                       'A. triostegus', 'E. areolatus', 'P. coelestis', 'A. stellatus',
                       'T. lunare', 'L. bohar', 'P. porosus', 'E. multinotatus']
display(df_fig1)

,region_attributes.label,all,train,test,val,scientific_name
0,fish,37573,30054,3753,3766,Fish
1,lethrinus_punctulatus,2170,1737,216,217,L. punctulatus
2,lutjanus_sebae,724,580,73,71,L. sebae
3,lutjanus_vitta,498,403,48,47,L. vitta
4,lethrinus_atkinsoni,425,340,43,42,L. atkinsoni
5,acanthurus_triostegus,230,189,21,20,A. triostegus
6,epinephelus_areolatus,191,149,22,20,E. areolatus
7,pomacentrus_coelestis,182,150,14,18,P. coelestis
8,abalistes_stellatus,173,134,19,20,A. stellatus
9,thalassoma_lunare,150,121,14,15,T. lunare


In [23]:
ShowClassSplits(df_top12_split_final, normalize=True, cat_col = 'region_attributes.label')

,all,train,test,val
region_attributes.label,,,,
fish,0.880239,0.880291,0.880159,0.879907
lethrinus_punctulatus,0.050838,0.050877,0.050657,0.050701
lutjanus_sebae,0.016961,0.016988,0.017120,0.016589
lutjanus_vitta,0.011667,0.011804,0.011257,0.010981
lethrinus_atkinsoni,0.009957,0.009959,0.010084,0.009813
acanthurus_triostegus,0.005388,0.005536,0.004925,0.004673
epinephelus_areolatus,0.004475,0.004364,0.005159,0.004673
pomacentrus_coelestis,0.004264,0.004394,0.003283,0.004206
abalistes_stellatus,0.004053,0.003925,0.004456,0.004673


In [78]:
import plotly.graph_objects as go

fig = go.Figure()

# x= ['Fish', 'L. punctulatus', 'L. sebae', 'L. vitta', 'L. atkinsoni', 
#     'A. triostegus', 'E. areolatus', 'P. coelestis', 'A. stellatus',
#     'T. lunare', 'L. bohar', 'P. porosus', 'E. multinotatus']

fig.add_trace(go.Bar(x=df_fig1["scientific_name"], y=df_fig1["train"], name='Train'))
fig.add_trace(go.Bar(x=df_fig1["scientific_name"], y=df_fig1["val"], name='Validation', marker_pattern_shape="+",))
fig.add_trace(go.Bar(x=df_fig1["scientific_name"], y=df_fig1["test"], name='Test', marker_pattern_shape="x"))

fig.update_traces(
    marker=dict(line_color="black", pattern_fillmode="overlay")
)

fig.update_layout(template="simple_white",
                  font_family="Times New Roman",
                  plot_bgcolor='rgba(0,0,0,0)',
                  font=dict(size=40),
                  title_font_family="Times New Roman",
                  title='Train, Validation and Test data split', title_x = 0.5,
                  yaxis=dict(
                      title='Number of Labelled Objects (log<sub>10</sub>)',
                      titlefont_size=40,
                      tickfont_size=34,
                      tickmode = 'linear',
                      tickformat = "digits",
                      title_font_family="Times New Roman"
                  ),
                  xaxis=dict(
                      title='Labels',
                      titlefont_size=40,
                      tickfont_size=34,  
                      title_font_family="Times New Roman"
                  ),
                  legend=dict(
                      xanchor="right",
                      yanchor="top",
                      bgcolor='rgba(255, 255, 255, 0)',
                      bordercolor='rgba(255, 255, 255, 0)',      
                      font=dict(
                          family="Times New Roman",
                          size=34,
                          color="black"
                      ),
                      itemsizing='constant'                      
                  ),
                  barmode='group',
                  xaxis_tickangle=-45,
                  autosize=True                  
    # bargap=0.15, # gap between bars of adjacent location coordinates.
    # bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.update_yaxes(type="log")
# fig.write_image(f"../datasets/fish/2022-15-05_df_top12_split_{SPLIT}-300dpi.png", width=7.086614*300, height=0.6*7.086614*300, scale=1)


### Leave One Video Out - Data Split

In [ ]:
from sklearn.model_selection import train_test_split

# prefix = ['A0', 'E0', 'G0']

train = ['G0', 'G0,E0', 'A0,E0']
test = ['A0,E0', 'A0', 'G0']

for TRAIN_PREFIX, TEST_PREFIX in zip(train,test):
    
    print(f'Train on prefix {TRAIN_PREFIX}')
    print(f'Test on prefix {TEST_PREFIX}')

    df_LVO = df_top12.copy()

    f_train = df_LVO[df_LVO.filename.str.contains(TRAIN_PREFIX.replace(",", "|"))]["filename"].unique()
    f_test = df_LVO[df_LVO.filename.str.contains(TEST_PREFIX.replace(",", "|"))]["filename"].unique()

    random.shuffle(f_train)
    random.shuffle(f_test)

    f_train, f_val = train_test_split(f_train, train_size=0.9, test_size=0.1, random_state=299, shuffle=True)

    print(f'train', len(f_train))
    print(f'validation', len(f_val))
    print(f'test', len(f_test))
    print(f'total frames',  len(f_train) + len(f_val) + len(f_test))

    df_LVO.loc[df_LVO.filename.isin(f_train), "split"] = "train"
    df_LVO.loc[df_LVO.filename.isin(f_val), "split"] = "val"
    df_LVO.loc[df_LVO.filename.isin(f_test), "split"] = "test"

    print("Convert YOLO BBox")
    df_LVO['yolo_fish'] = df_LVO.apply(lambda row : rect_to_yolo(row['fish'],
                                                                 row['shape_attributes.x'],
                                                                 row['shape_attributes.y'],
                                                                 row['shape_attributes.width'],
                                                                 row['shape_attributes.height']
                                                                ), axis = 1)

    ff = TRAIN_PREFIX.replace(",", "-")
    # df_LVO.to_csv(f"../datasets/df_LVO_{ff}.csv")
    print()

    # Check if we can do LVO at species level
    for f in df_LVO.filename.unique():

        if df_LVO[df_LVO.filename == f]['split'].nunique() != 1:
            print('something is not right here:', f)

    df_fig1 = ShowClassSplits(df_LVO, normalize=False, cat_col = 'region_attributes.label')
    df_fig1 = df_fig1.reset_index()
    display(df_fig1)

# DeepFish Dataset
- https://github.com/alzayats/DeepFish
- Classification: only empty (no fish) and valid (fish) images
- Localization: (x,y) coordinate in a mask .PNG format. empty (no fish) and valid (fish) - every 10 consecutive frame

In [ ]:
deepfish_clf = pd.read_csv("../datasets/deepfish/Classification/classification.csv")
deepfish_loc = pd.read_csv("../datasets/deepfish/Localization/Localization.csv")

In [ ]:
deepfish_clf.head()

In [ ]:
deepfish_clf.labels.sum()

In [ ]:
deepfish_clf.classes.value_counts()

In [ ]:
deepfish_loc.head()

In [ ]:
deepfish_loc.counts.sum()

In [ ]:
deepfish_loc.classes.value_counts()